In [1]:
import os
import pandas as pd

In [2]:
external_df = pd.read_csv('../external_data/external_df.csv', index_col=False)

In [4]:
external_df.head()

,사용일자,중분류_카운트,중분류
0,2020-01-01,2509091,버스
1,2020-01-02,5286628,버스
2,2020-01-03,5580078,버스
3,2020-01-04,4036695,버스
4,2020-01-05,3138649,버스


In [7]:
print(external_df.shape)

(546, 3)
